# RAG - FSI FAQ with FAISS Local VectorStore Test
이 작업(TASK)에서는 앞서 생성한 SageMaker Embedding Vector 모델의 Endpoint를 활용하여 로컬 환경에서 자주 묻는 질문(FAQ) 데이터셋에 대한 FAISS(Facebook AI Similarity Search) 임베딩을 생성합니다. 이를 통해 텍스트 데이터의 유사성을 빠르게 검색할 수 있게 됩니다. 더불어, SageMaker에서 제공하는 Language Model for Learning-to-Match (LLM) Endpoint를 사용하여, 임베딩으로 검색된 정보를 기반으로 프롬프트 엔지니어링을 수행합니다. 이 과정을 통해 Retriever-Answer Generator (RAG) 로직을 테스트해봅니다.

In [3]:
%store -r endpoint_name_emb endpoint_name_text
try:
    endpoint_name_emb
    endpoint_name_text
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] TASK-1, TASK-2 노트북을 다시 실행해 주세요.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

## 1. SageMaker Endpoint Wrapper

In [4]:
# # Version fixed management during code stabilization period of langchain
# !pip install SQLAlchemy==2.0.1

In [5]:
import sys
import IPython
%load_ext autoreload
%autoreload 2
sys.path.append('./utils') # src 폴더 경로 설정

import json
import boto3
import numpy as np
from inference_utils import Prompter
from typing import Any, Dict, List, Optional
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.llms.sagemaker_endpoint import LLMContentHandler, SagemakerEndpoint
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler

### 1.1 [한국어 Text LLM] SageMaker Kullum endpiont handler
- LLM Kullum Endpoint의 RequestHandler 함수를 만듭니다.

In [6]:
prompter = Prompter("kullm")
params = {
      'do_sample': False,
      'max_new_tokens': 512, #128
      'temperature': 1.0,  # 0.5 ~ 1.0 default = 1.0 높으면 랜덤하게 자유도. 다음 생성 문장 토큰의 자유도 
      'top_k': 0,
      'top_p': 0.9,
      'return_full_text': False,
      'repetition_penalty': 1.1,
      'presence_penalty': None,
      'eos_token_id': 2
}

class KullmContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        '''
        입력 데이터 전처리 후에 리턴
        '''
        context, question = prompt.split("||SPEPERATOR||") 
        prompt = prompter.generate_prompt(question, context)

        #print ("prompt", prompt)
        payload = {
            'inputs': [prompt],
            'parameters': model_kwargs
        }
                           
        input_str = json.dumps(payload)
        
        return input_str.encode('utf-8')
    

    def transform_output(self, output: bytes) -> str:
        
        response_json = json.loads(output.read().decode("utf-8"))              
        generated_text = response_json[0][0]["generated_text"]
        
        return generated_text    

#### 1.1.a 기존에 deploying한 한국어생성 LLM endpoint 이름을 설정합니다. 
- endpoint_name_text

In [8]:
aws_region = boto3.Session().region_name
LLMTextContentHandler = KullmContentHandler()
seperator = "||SPEPERATOR||"

llm_text = SagemakerEndpoint(
    endpoint_name=endpoint_name_text,
    region_name=aws_region,
    model_kwargs=params,    
    content_handler=LLMTextContentHandler,
)

### 1.2 [한국어 임베딩벡터 모델] SageMaker 임베딩 벡터 모델 KoSimCSE-roberta endpiont handler
- KoSimCSE-roberta

### SagemakerEndpointEmbeddingsJumpStart클래스는 SagemakerEndpointEmbeddings를 상속받아서 작성

매개변수 (Parameters):
* texts: 임베딩을 생성할 텍스트의 리스트입니다.
* chunk_size: 한 번의 요청에 그룹화될 입력 텍스트의 수를 정의합니다. 만약 None이면, 클래스에 지정된 청크 크기를 사용합니다.

Returns:
* 각 텍스트에 대한 임베딩의 리스트를 반환

In [9]:
class SagemakerEndpointEmbeddingsJumpStart(SagemakerEndpointEmbeddings):
    def embed_documents(self, texts: List[str], chunk_size: int=1) -> List[List[float]]:
        """Compute doc embeddings using a SageMaker Inference Endpoint.

        Args:
            texts: The list of texts to embed.
            chunk_size: The chunk size defines how many input texts will
                be grouped together as request. If None, will use the
                chunk size specified by the class.

        Returns:
            List of embeddings, one for each text.
        """
        results = []
        _chunk_size = len(texts) if chunk_size > len(texts) else chunk_size
        
        print("text size: ", len(texts))
        print("_chunk_size: ", _chunk_size)

        for i in range(0, len(texts), _chunk_size):
            
            #print (i, texts[i : i + _chunk_size])
            response = self._embedding_func(texts[i : i + _chunk_size])
            #print (i, response, len(response[0].shape))
            
            results.extend(response)
        return results

In [10]:
class KoSimCSERobertaContentHandler(EmbeddingsContentHandler):
    
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        
        input_str = json.dumps({"inputs": prompt, **model_kwargs})
        
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        
        response_json = json.loads(output.read().decode("utf-8"))
        ndim = np.array(response_json).ndim    
        
        if ndim == 4:
            # Original shape (1, 1, n, 768)
            emb = response_json[0][0][0]
            emb = np.expand_dims(emb, axis=0).tolist()
        elif ndim == 2:
            # Original shape (n, 1)
            emb = []
            for ele in response_json:
                e = ele[0][0]
                emb.append(e)
        else:
            print(f"Other # of dimension: {ndim}")
            emb = None
        return emb

#### 1.2.a 기존에 deploying한 임베딩 벡터 endpoint 설정

- endpoint_name_emb

In [11]:
LLMEmbHandler = KoSimCSERobertaContentHandler()

llm_emb = SagemakerEndpointEmbeddingsJumpStart(
    endpoint_name=endpoint_name_emb,
    region_name=aws_region,
    content_handler=LLMEmbHandler,
)

**Now, we can build an QA application. <span style="color:red">LangChain makes it extremly simple with following few lines of code</span>.**

# 2.Vector Store - FAISS

### FAISS(Facebook AI Similarity Search) : 
- 밀집 벡터의 효율적인 유사성 검색과 클러스터링을 위한 라이브러리입니다. 이 라이브러리는 RAM에 들어가지 않을 수도 있는 어떤 크기의 벡터 세트에서도 검색할 수 있는 알고리즘을 포함하고 있습니다.
- CPU(pip install faiss-cpu)와 GPU(pip install faiss-gpu) 모두에서 사용할 수 있습니다.
- 유사성 검색: 기본적인 유사성 검색 외에도, 유사성 점수를 반환하는 similarity_search_with_score와 같은 FAISS 특화 메서드가 있습니다.
- 벡터 기반 검색: similarity_search_by_vector 메서드를 사용하여 임베딩 벡터를 파라미터로 전달할 수 있습니다.
- 저장 및 로딩: FAISS 인덱스를 로컬에 저장하고 로딩하는 기능이 있어, 매번 인덱스를 생성할 필요가 없습니다.
- 벡터 스토어 병합: 두 개의 FAISS 벡터 스토어를 병합할 수 있습니다.
- 필터링 지원: FAISS는 메타데이터를 기반으로 문서를 필터링하는 기능을 제공합니다.
- 직렬화와 역직렬화: FAISS 인덱스를 바이트로 직렬화하고 역직렬화할 수 있습니다, 이는 데이터베이스에 인덱스를 저장할 때 유용합니다.

### Vector Store :
- Vector Store는 비정형 데이터를 임베딩 벡터로 변환하여 저장하고, 쿼리 시에 해당 임베딩 벡터와 '가장 유사한' 임베딩 벡터를 검색하는 역할을 하는 저장소입니다. 이는 텍스트 임베딩 모델과 연계하여 벡터를 생성하고 저장, 검색하는 기능을 제공합니다. 다양한 벡터 저장 옵션들이 있으며, 비동기 작업도 지원합니다.


References
- Vector Store : https://python.langchain.com/docs/modules/data_connection/vectorstores/
- FAISS Vector Store : https://python.langchain.com/docs/modules/data_connection/vectorstores/integrations/faiss



In [12]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import Chroma, AtlasDB, FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

### 2.1 Context file load

#### 2.1.a Increase csv field size

In [13]:
import csv
csv.field_size_limit(100000000)
csv.field_size_limit()

100000000

#### 2.1.b FAQ CSV 파일 Load

In [14]:
loader = CSVLoader(
    file_path="./dataset/fsi_smart_faq_ko.csv",
    source_column="Source",
    encoding="utf-8"
)
context_documents = loader.load()

#### 2.1.c FSI FAQ Context 데이터 확인 
- 인터넷뱅킹 FAQ > 스마트뱅킹 No.1 ~ N. 89 로 구성되었습니다. 
- https://www.shinhan.com/hpe/index.jsp#050101020000

In [15]:
print(len(context_documents))

context_documents[0:3]

89


[Document(page_content='no: 89\nCategory: 타기관OTP 이용등록방법 알려주세요\nInformation: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \n[경로]\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \n- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\n \n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\n\n기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.\ntype: 인터넷뱅킹\nSource: 신한은행', metadata={'source': '신한은행', 'row': 0}),
 Document(page_content='no: 88\nCategory: 공동인증서와 금융인증서 차이점이 무엇인가요?\nInformation: 공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자거래범용(수수료 4,400원) 인증서가 있으며 유효기간은 1년입니다. \n공동인증서는 하드디스크나 이동식디스크, 휴대폰 등 원하시는 기기에 저장해서 이용할 수 있습니다.\n인증서를 저장한 매체에서는 인증서 비밀번호로 편리하게 이용할 수 있으나 다른 기기에서 이용하려면 기기마다 복사하거나 이동식디스크에 저장해서 휴대해야 하는 불편함이 있을 수 있습니다.\n\n금융인증서는 금융인증서는 금융결제원의 클라우드에 저장하여 이용하는 인증서로 발급/이용 시에 클라우드에 접속이 필요합니다.\n금융결제원 클라우드에 연결만 가능하다면 어디서든 편리하게 이용 가능하지만, PC나 USB, 휴대폰 등 다른 기기로 복사는 불가합니다.(유효기간 3년/발급 수수료 무료)\n※ 클라우드 계정 연결을 위해 휴대폰을 통한 ARS, SMS, 마이인포앱 인증 절차가 필요합니다.\ntype: 인증서\nSource: 신한은행', metadata={'source': '신한은행', 'row': 1}),

### 2.2 FAISS 벡터 Indexer 생성

로드한 FAQ 컨텍스트 데이터를 Indexer를 생성 빌드합니다.
- Recursively split by character : https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter

It takes about 1 mins
vectorstore 생성에 1분정도 소요됩니다.
Document가 부분 절삭 되지 않도록 Chunking 전략을 잘 세웁니다.

In [16]:
index_creator = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding=llm_emb,
    #text_splitter=CharacterTextSplitter(chunk_size=700, chunk_overlap=0),
    #text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200), 
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
)
# 로버타 모델의 청크 크기에 종속되어있음. 청크 파라미터 확인

### 2.2.a 임베딩 벡터 스토어 파일 생성 시작

In [17]:
%%time
index = index_creator.from_loaders([loader])

text size:  90
_chunk_size:  1
CPU times: user 4.83 s, sys: 150 ms, total: 4.98 s
Wall time: 15 s


### 2.2.b index save and read index from local

In [18]:
# Save
index.vectorstore.save_local("./indexer/fsi_faq_indexer_ko")

In [19]:
# Read
index_ = FAISS.load_local("./indexer/fsi_faq_indexer_ko", llm_emb, allow_dangerous_deserialization=True)

### 2.2.c 생성한 Indexer document 5개 확인

In [20]:
index_.index_to_docstore_id
#index_.docstore._dict[:10]
dict(list(index_.docstore._dict.items())[:5])

{'55079f7e-64fe-4fd2-9f9d-4cb081fbd057': Document(page_content='no: 89\nCategory: 타기관OTP 이용등록방법 알려주세요\nInformation: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \n[경로]\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \n- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\n \n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\n\n기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.\ntype: 인터넷뱅킹\nSource: 신한은행', metadata={'source': '신한은행', 'row': 0}),
 'd86253da-ff52-4b11-b307-bca96b3daeea': Document(page_content='no: 88\nCategory: 공동인증서와 금융인증서 차이점이 무엇인가요?\nInformation: 공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자거래범용(수수료 4,400원) 인증서가 있으며 유효기간은 1년입니다. \n공동인증서는 하드디스크나 이동식디스크, 휴대폰 등 원하시는 기기에 저장해서 이용할 수 있습니다.\n인증서를 저장한 매체에서는 인증서 비밀번호로 편리하게 이용할 수 있으나 다른 기기에서 이용하려면 기기마다 복사하거나 이동식디스크에 저장해서 휴대해야 하는 불편함이 있을 수 있습니다.\n\n금융인증서는 금융인증서는 금융결제원의 클라우드에 저장하여 이용하는 인증서로 발급/이용 시에 클라우드에 접속이 필요합니다.\n금융결제원 클라우드에 연결만 가능하다면 어디서든 편리하게 이용 가능하지만, PC나 USB, 휴대폰 등 다른 기기로 복사는 불가합니다.(유효기간 3년/발급 수수료 무료)\n※ 클라우드 계정 연결을 위해 휴대폰을 통한 ARS, SMS, 마이인포앱 

In [21]:
len(list(index_.docstore._dict.items()))

90

## 3. Langchain QnA

In [22]:
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

### 3.1 Query and Response

filter_and_remove_score
- 이 함수는 주어진 검색 결과 리스트(res)에서 특정 점수(cutoff_score와 variance를 기반으로)를 기준으로 필터링을 수행하고, 임베딩 Document 이외 검색 결과에 추가된 score를 제거합니다.
- 본 핸즈온 데모에서는 카테고리 분류기 대신, 단순 검색 조건 필터링에 조회되지 않는 에러가 큰 (에러 거리 score가 200이상이고, 분산이 1.3이상인 값을 상한값으로 정한 다음 필터링된 문서 리스트만 반환합니다.
- 실제 환경에서는 문장 분류 카테고리 모델의 사용을 권장합니다. 



In [23]:
prompt_template = ''.join(["{context}", seperator, "{question}"])
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain = load_qa_chain(llm=llm_text, chain_type="stuff", prompt=PROMPT, verbose=True)


def filter_and_remove_score(res, cutoff_score = 200, variance=1.3):
    # Get the lowest score
    lowest_score = min(score for doc, score in res)
    print('lowest_score : ', lowest_score)
    # If the lowest score is over 200, return an empty list
    if lowest_score > cutoff_score:
        return []
    # Calculate the upper bound for scores
    upper_bound = lowest_score * variance

    
    # Filter the list and remove the score
    res = [doc for doc, score in res if score <= upper_bound]

    return res


def get_similiar_docs(query, k=1, fetch_k=50, score=True, bank=""):

    #print (query)
    
    if score:         
        similar_docs = index_.similarity_search_with_score(
            query,
            k=k,
            fetch_k=fetch_k,
            filter=dict(source=bank)
        )
        similar_docs=filter_and_remove_score(similar_docs)
    else:        
        similar_docs = index_.similarity_search(
            query,
            k=k,
            fetch_k=fetch_k,
            filter=dict(source=bank)
        )
        
    return similar_docs

def get_answer(query, bank="", k=1):
                
    search_query = query
    
    similar_docs = get_similiar_docs(search_query, k=k, bank=bank)
    
    print("similar_docs : ", similar_docs)
    llm_query = ''+query+' Category에 대한 Information을 찾아서 설명해주세요.'
    
    if not similar_docs:
        llm_query = query

    answer = chain.run(input_documents=similar_docs, question=llm_query)
    
    return answer



def get_search_answer (query, bank="", k=1):
    search_query = query
    similar_docs = get_similiar_docs(search_query, k=k, bank=bank)
    llm_query = '검색된 내용을 no. 별로 요약한 문장으로 알려줘.'
    if not similar_docs:
        llm_query = query

    llm_answer = chain.run(input_documents=similar_docs, question=llm_query)
    return llm_answer

In [24]:
chain.run(input_documents='', question='안녕하세요')



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
||SPEPERATOR||안녕하세요

> Finished chain.

> Finished chain.


'안녕하세요! 오늘은 무엇을 도와드릴까요?'

In [25]:
%%time
question = "안녕하세요. 날씨가 참 좋네요."
response = get_answer(question, bank="신한은행", k=5)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')

lowest_score :  293.9933
similar_docs :  []


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
||SPEPERATOR||안녕하세요. 날씨가 참 좋네요.

> Finished chain.

> Finished chain.
question: 안녕하세요. 날씨가 참 좋네요.


response: 
 안녕하세요! 오늘은 무엇을 도와드릴까요?
CPU times: user 8.26 ms, sys: 4.57 ms, total: 12.8 ms
Wall time: 674 ms


### 3.1.a FAQ 데이터 응답문장 테스트

In [26]:
%%time
question = "타기관OTP 등록 방법 알려주세요"
response = get_answer(question, bank="신한은행", k=3)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')

lowest_score :  112.327644
similar_docs :  [Document(page_content='no: 89\nCategory: 타기관OTP 이용등록방법 알려주세요\nInformation: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \n[경로]\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \n- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\n \n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\n\n기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.\ntype: 인터넷뱅킹\nSource: 신한은행', metadata={'source': '신한은행', 'row': 0})]


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
no: 89
Category: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  
- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록
 
 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
type: 인터넷뱅킹
Source: 신한은행||SPEPERATOR||타기관OTP 등록 방법 알려주세요 Category에 대한 Information을 찾아서 설명해주세요.

> Finished chain.

> Finished chain.
question: 타기관OTP 등록 방법 알려주세요


response: 
 타기관

In [27]:
%%time
question = "홈페이지 이용자아이디 여러 개 사용할 수 있나요?"
response = get_answer(question, bank="신한은행", k=3)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')

lowest_score :  91.2802
similar_docs :  [Document(page_content='no: 71\nCategory: 홈페이지 이용자아이디 여러 개 사용할 수 있나요?\nInformation: 홈페이지 이용자 아이디는 개인의 경우 1인 1개만 이용 가능하고 기업의 경우에는 1개의 사업자번호당 사용자별로 이용자아이디를 복수로 사용할 수 있습니다. \n※ 개인사업자의 경우 개인과 기업 각각 이용자아이디를 발급하여 복수로 이용 가능합니다. \n기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.\ntype: 홈페이지\nSource: 신한은행', metadata={'source': '신한은행', 'row': 18})]


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
no: 71
Category: 홈페이지 이용자아이디 여러 개 사용할 수 있나요?
Information: 홈페이지 이용자 아이디는 개인의 경우 1인 1개만 이용 가능하고 기업의 경우에는 1개의 사업자번호당 사용자별로 이용자아이디를 복수로 사용할 수 있습니다. 
※ 개인사업자의 경우 개인과 기업 각각 이용자아이디를 발급하여 복수로 이용 가능합니다. 
기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
type: 홈페이지
Source: 신한은행||SPEPERATOR||홈페이지 이용자아이디 여러 개 사용할 수 있나요? Category에 대한 Information을 찾아서 설명해주세요.

> Finished chain.

> Finished chain.
question: 홈페이지 이용자아이디 여러 개 사용할 수 있나요?


response: 
 예, 홈페이지 이용자 아이디는 개인의 경우 1인 1개만 사용할 수 있고, 기업의 경우 1개의 사업자번호당 사용자별로 이용자아이디를 복수로 사

In [28]:
%%time
question = "화면에서 미래를함께하는따뜻한금융 안전한금융거래를위해준비중입니다. 라고 나오고 맨날 멈추는데 어떻게 하냐? 아니쓸수가 없자나"
response = get_answer(question, bank="신한은행", k=1)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')

lowest_score :  180.9096
similar_docs :  [Document(page_content='no: 78\nCategory: 미래를함께하는따뜻한금융 안전한금융거래를위해준비중입니다.\' 화면에서 멈춤 / 로딩중 멈추는데 어떻게 하나요?\nInformation: 오류 사유는 인터넷 익스플로러 설정값의 영향이거나 이용하시는 장소의 네트워크 영향에 의해 로딩이 원활하지 않은 경우입니다. \n조치방법은 \n1. 임시 인터넷 파일 및 쿠키 삭제, 신뢰할 수 있는 사이트 추가 \n 인터넷 익스플로러 상단의 [도구] > [인터넷옵션]에서\n① [일반Tab] > 검색기록항목의 [삭제] 버튼 클릭 "임시 인터넷 파일 및 사이트 파일" , "쿠키 및 웹사이트 데이터" 항목만 체크하고 [삭제] \n② [보안Tab] > 신뢰할 수 있는 사이트 > [사이트] 클릭하여 [ https://*.shinhan.com ] 추가 후 적용 및 확인하십시오. \n 또는, 당행 보안프로그램 삭제 열려있는 브라우저 종료 후, 시작 > 제어판의 [프로그램 추가/제거] (총 4개이며 보이지 않으면 제거할 필요없음) \n- AhnLab Safe Transaction, \n - iniLINE CrossEX Service, \n- INISAFE CrossWeb EX v3.0, \n- TouchEn nxKey with E2E for 32bit\n3. 새 브라우저 열어서 개인인터넷뱅킹 재접속 후 [통합설치] 진행 하세요. \n기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.\ntype: \nSource: 신한은행', metadata={'source': '신한은행', 'row': 11})]


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
no: 78
Category: 미래를함께하는따뜻한금융 안전한금융거래를위해준비중입니다

In [29]:
%%time
question = "공동인증서랑 금융인증서 같이 쓸수있나요?"
response = get_answer(question, bank="신한은행", k=3)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')

lowest_score :  85.21392
similar_docs :  [Document(page_content='no: 84\nCategory: 기존 공동인증서를 보유한 상태에서 금융인증서 발급이 가능한가요?\nInformation: 공동인증서와 금융인증서는 별개의 인증서로 두 가지 인증서를 모두 사용할 수 있습니다.\ntype: 금융인증서\nSource: 신한은행', metadata={'source': '신한은행', 'row': 5})]


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
no: 84
Category: 기존 공동인증서를 보유한 상태에서 금융인증서 발급이 가능한가요?
Information: 공동인증서와 금융인증서는 별개의 인증서로 두 가지 인증서를 모두 사용할 수 있습니다.
type: 금융인증서
Source: 신한은행||SPEPERATOR||공동인증서랑 금융인증서 같이 쓸수있나요? Category에 대한 Information을 찾아서 설명해주세요.

> Finished chain.

> Finished chain.
question: 공동인증서랑 금융인증서 같이 쓸수있나요?


response: 
 예, 공동인증서와 금융인증서는 별개의 인증서로, 공동인증서를 보유한 상태에서도 금융인증서를 발급받을 수 있습니다. 이 경우, 기존 공동인증서를 금융인증서로 전환하여 사용하면 됩니다.
CPU times: user 10.9 ms, sys: 170 µs, total: 11 ms
Wall time: 2.51 s


In [32]:
%%time
question = "홈페이지 탈퇴하는방법 알려줘"
response = get_answer(question, bank="신한은행", k=3)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')

lowest_score :  131.74153
similar_docs :  [Document(page_content='no: 48\nCategory: 홈페이지 탈회하고 싶습니다. 어떻게 하나요?\nInformation: 홈페이지 로그인 후 회원탈퇴를 하실 수 있습니다. 홈페이지 상단에 있는 고객센터 > 회원서비스 > 회원탈퇴 메뉴에서 회원탈퇴 진행할 수 있습니다. 혹시, 비밀번호를 분실하셨다면 ID/PW 로그인 화면 하단 비밀번호 재설정 메뉴를 통해서 비밀번호 재설정하고 로그인 후 회원탈퇴 가능합니다. 기타 문의는 신한은행고객센터 (☎ 1599-8000번)으로 문의 바랍니다.\ntype: 홈페이지\nSource: 신한은행', metadata={'source': '신한은행', 'row': 41}), Document(page_content='no: 17\nCategory: 인터넷뱅킹 해지 방법 알려주세요\nInformation: [경로] - 인터넷뱅킹 로그인 → 사용자관리 → 인터넷뱅킹관리 → 인터넷뱅킹해지 ☞ 신한은행에서 발급한 인증서 폐기여부 선택가능합니다. ☞ 쏠(모바일뱅킹) 도 사용할수 없음 - 신한 쏠(SOL) 로그인 → 전체메뉴 → 설정/인증 → 서비스 해지 ☞ 쏠(SOL)만 해지 또는 인터넷뱅킹&쏠(SOL) 모두 해지 중 선택하여 해지가능합니다. 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.\ntype: 인터넷뱅킹\nSource: 신한은행', metadata={'source': '신한은행', 'row': 72}), Document(page_content='no: 24\nCategory: 인터넷뱅킹 자동이체해지 방법안내\nInformation: 인터넷뱅킹에 로그인하신후, 좌측의 메뉴항목에서 이체 > 자동이체 > 자동이체조회 변경/취소 서비스를 통해 해지하실 수 있습니다.\ntype: 인터넷뱅킹\nSource: 신한은행', metadata={'source': '신한은행', 'row': 65})]


> Enterin

In [33]:
%%time
question = "홈페이지 탈퇴하는방법 알려줘"
response = get_search_answer(question, bank="신한은행", k=3)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')

lowest_score :  131.74153


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
no: 48
Category: 홈페이지 탈회하고 싶습니다. 어떻게 하나요?
Information: 홈페이지 로그인 후 회원탈퇴를 하실 수 있습니다. 홈페이지 상단에 있는 고객센터 > 회원서비스 > 회원탈퇴 메뉴에서 회원탈퇴 진행할 수 있습니다. 혹시, 비밀번호를 분실하셨다면 ID/PW 로그인 화면 하단 비밀번호 재설정 메뉴를 통해서 비밀번호 재설정하고 로그인 후 회원탈퇴 가능합니다. 기타 문의는 신한은행고객센터 (☎ 1599-8000번)으로 문의 바랍니다.
type: 홈페이지
Source: 신한은행

no: 17
Category: 인터넷뱅킹 해지 방법 알려주세요
Information: [경로] - 인터넷뱅킹 로그인 → 사용자관리 → 인터넷뱅킹관리 → 인터넷뱅킹해지 ☞ 신한은행에서 발급한 인증서 폐기여부 선택가능합니다. ☞ 쏠(모바일뱅킹) 도 사용할수 없음 - 신한 쏠(SOL) 로그인 → 전체메뉴 → 설정/인증 → 서비스 해지 ☞ 쏠(SOL)만 해지 또는 인터넷뱅킹&쏠(SOL) 모두 해지 중 선택하여 해지가능합니다. 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
type: 인터넷뱅킹
Source: 신한은행

no: 24
Category: 인터넷뱅킹 자동이체해지 방법안내
Information: 인터넷뱅킹에 로그인하신후, 좌측의 메뉴항목에서 이체 > 자동이체 > 자동이체조회 변경/취소 서비스를 통해 해지하실 수 있습니다.
type: 인터넷뱅킹
Source: 신한은행||SPEPERATOR||검색된 내용을 no. 별로 요약한 문장으로 알려줘.

> Finished chain.

> Finished chain.
question: 홈페이지 탈퇴하는방법 알려줘


response: 
 1. 홈페이지 로그

In [34]:
%%time
question = "질문 답변 감사합니다. 좋은 하루 되세요."
response = get_answer(question, bank="신한은행", k=3)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')

lowest_score :  236.92528
similar_docs :  []


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
||SPEPERATOR||질문 답변 감사합니다. 좋은 하루 되세요.

> Finished chain.

> Finished chain.
question: 질문 답변 감사합니다. 좋은 하루 되세요.


response: 
 천만에요! 도움을 드릴 수 있어서 기쁩니다. 다른 질문이 있으시면 언제든지 문의해 주세요.
CPU times: user 9.27 ms, sys: 78 µs, total: 9.35 ms
Wall time: 1.31 s
